# Graph Algorithms

## Undirected Graph

In [1]:
from typing import TypeVar, List, Dict, Tuple
T = TypeVar("T")

class Graph:
    """
    An undirected Graph with adjacency lists (dictionary)
    """

    def __init__(self):
        """
        Constructor
        """
        self._vertices: Dict[T, List[T]] = {}

    def add_edge(self, v: T, u: T) -> None:
        """
        Add an edge between v and u to the graph
        """
        if v not in self._vertices:
            self._vertices[v] = []
        if u not in self._vertices:
            self._vertices[u] = []

        self._vertices[v].append(u)
        self._vertices[u].append(v)

    def get_edges(self, v:T):
        """
        Return the list of edges for vertex v 
        """
        return self._vertices[v]
    
    def get_vertices(self) -> List[T]:
        """
        Return the list of the vertices in the graph
        """
        return self._vertices.keys()
    
    def edge_list(self) -> List[Tuple[T,T]]:
        """
        Return the list of edges in the graph as a list of tuples. 
        """
        l = []
        for v in self._vertices:
            for e in self._vertices[v]:
                l.append((v, e))
        return l
    
    def __str__(self):
        """
        Pretty print the adjacency lists of the graph.
        """
        r = ""
        for u in self.get_vertices():
            r += "{0} --> [".format(u)
            for v in self.get_edges(u):
                r+= "{0}, ".format(v)
            r += '{0}{0}]\n'.format(chr(0x8))
        return r

## Example graph of US cities

![USA](images/USA-unweighted.png)

In [2]:
# Build the Graph of US cities
def build_cities_graph():
    cities = Graph()

    cities.add_edge("Seattle", "Chicago")
    cities.add_edge("Seattle", "San Francisco")
    cities.add_edge("San Francisco", "Riverside")
    cities.add_edge("San Francisco", "Los Angeles")
    cities.add_edge("Los Angeles", "Riverside")
    cities.add_edge("Los Angeles", "Phoenix")
    cities.add_edge("Riverside", "Phoenix")
    cities.add_edge("Riverside", "Chicago")
    cities.add_edge("Phoenix", "Dallas")
    cities.add_edge("Phoenix", "Houston")
    cities.add_edge("Dallas", "Chicago")
    cities.add_edge("Dallas", "Atlanta")
    cities.add_edge("Dallas", "Houston")
    cities.add_edge("Houston", "Atlanta")
    cities.add_edge("Houston", "Miami")
    cities.add_edge("Atlanta", "Chicago")
    cities.add_edge("Atlanta", "Washington")
    cities.add_edge("Atlanta", "Miami")
    cities.add_edge("Miami", "Washington")
    cities.add_edge("Chicago", "Detroit")
    cities.add_edge("Detroit", "Boston")
    cities.add_edge("Detroit", "Washington")
    cities.add_edge("Detroit", "New York")
    cities.add_edge("Boston", "New York")
    cities.add_edge("New York", "Philadelphia")
    cities.add_edge("Philadelphia", "Washington")
    return cities

In [3]:
g = build_cities_graph()
print(g) # use __str__ method to pretty print adjacency lists

Seattle --> [Chicago, San Francisco]
Chicago --> [Seattle, Riverside, Dallas, Atlanta, Detroit]
San Francisco --> [Seattle, Riverside, Los Angeles]
Riverside --> [San Francisco, Los Angeles, Phoenix, Chicago]
Los Angeles --> [San Francisco, Riverside, Phoenix]
Phoenix --> [Los Angeles, Riverside, Dallas, Houston]
Dallas --> [Phoenix, Chicago, Atlanta, Houston]
Houston --> [Phoenix, Dallas, Atlanta, Miami]
Atlanta --> [Dallas, Houston, Chicago, Washington, Miami]
Miami --> [Houston, Atlanta, Washington]
Washington --> [Atlanta, Miami, Detroit, Philadelphia]
Detroit --> [Chicago, Boston, Washington, New York]
Boston --> [Detroit, New York]
New York --> [Detroit, Boston, Philadelphia]
Philadelphia --> [New York, Washington]



### Graphviz graph visualization module

Sometimes it is helpful to be able to draw the graph using [Graphviz](https://graphviz.readthedocs.io/)

Need to do the following

```
conda install graphviz
conda install pytho-graphviz
```

In [4]:
import graphviz

def g2gviz(g: Graph, fn: str) -> None:
    """
    A function to return a graphviz graph from one of our Graph objects
    """
    if isinstance(g,DiGraph):
        print("generating graphviz Digraph")
        gv = graphviz.Digraph(format = 'png', filename = fn)
    elif isinstance(g, Graph):
        print("generating graphviz Graph")
        gv = graphviz.Graph(format = 'png', filename = fn)
        
    edges = g.edge_list()
    nodups = []
    
    # remove the duplicate edges one of (u,v) or (v,u)
    for (u,v) in edges:
        if (v,u) not in nodups:
            nodups.append((u,v))
        
    for (u,v) in nodups:
        gv.edge(u,v)
        
    return gv

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
gv = g2gviz(g,'states')
gv.render(filename='states.dot')

<img src = "states.dot.png" width="50%" />

## Depth First Search (DFS)

Keep track of pre and post visit numbers

In [ ]:
def dfs(g: Graph, v:T) -> Tuple[List[T], Dict[T,int], Dict[T, int]]:

    def explore(v:T):
        nonlocal clock # clock is defined in the outer scope
        visited[v] = True
        order.append(v)
        pre[v] = clock
        clock += 1
        
        for e in g.get_edges(v):
            if e not in visited:
                explore(e)
                
        post[v] = clock
        clock += 1

    # The main part of the DFS function
    order = []        # The DFS order
    visited = dict()  # The visited dictionary
    pre = dict()      # prevsit number dictionary
    post = dict()     # postvisit number dictionary
    clock = 0         # counter for numbers pre/post visits
    
    explore(v)
    
    for v in g.get_vertices():
        if v not in visited:
            explore(v)
    return order, pre, post

In [ ]:
# Example graph from exercise 3.1 graph 
g31 = Graph()  
g31.add_edge('a','b')
g31.add_edge('a','e')
g31.add_edge('b','c')
g31.add_edge('b','e')
g31.add_edge('c','f')
g31.add_edge('e','f')
g31.add_edge('f','i')
g31.add_edge('d', 'g')
g31.add_edge('g', 'h')
g31.add_edge('h', 'd')
order, pre, post = dfs(g31, 'a')
print(order)
print(pre)
print(post)

In [ ]:
gv = g2gviz(g31,'exercise31.png')
gv

## Directed Graphs

Inherits the graph class and override the `add_edge` function so we just add an edge in one direction

In [ ]:
class DiGraph(Graph):

    def __init__(self):
        super().__init__()

    def add_edge(self, v:T, u:T):
        if v not in self.get_vertices():
            self._vertices[v] = []
        if u not in self._vertices:
            self._vertices[u] = []

        self._vertices[v].append(u)  # only append in one direction
  

A helper function to read a file of edges and return the DiGraph. 

In [ ]:
def build_digraph(fn: str, sep=' ') -> DiGraph:
    f = open(fn)
    dg = DiGraph()
    for line in f:
        (u,v) = line.strip().split(sep)
        dg.add_edge(u,v)
    
    return dg


## DFS with Stack

Replace recursion using a stack 

In [ ]:
def dfs(g: Graph, v:T):
    order = []
    visited = { }
    s = []         # use a list as a stack
    s.append(v)    # initialize stack with starting vertex 

    while len(s) > 0:
        curr = s.pop()
        
        if curr not in visited:
            visited[curr] = True
            order.append(curr)
            for e in g.get_edges(curr):
                if e not in visited:
                    s.append(e)
    return order



## Breadth First Search (BFS)

Breadth First Search use a queue. We use a double ended queue called `deque` pronounced _deck_.  Ntice this is the same as the Depth First Search where we just replace the stack with a queue.

In [ ]:
from collections import deque

def bfs(g: Graph, v:T) -> Tuple[List[T], Dict[T,T]]:
    order = []       # The breadth first order
    visited = { }    # te visited dictionary
    q = deque()      # the queue
    q.appendleft(v)  # initialize the queue with the starting vertex
    prev = dict()    # the previous dictionary for reconstructing paths

    while len(q) > 0:
        curr = q.pop()          # removes item in queue
        if curr not in visited:
            visited[curr] = True
            order.append(curr)
            for e in g.get_edges(curr):
                if e not in visited:
                    q.appendleft(e)
                    prev[e] = curr
    return order, prev

## Example Digraph 

In [ ]:
dg = build_digraph('dg.txt')
gv = g2gviz(dg,'states')
gv.render(filename='digraph_example.dot')

Here is the image we've been using in the class and lectures for the above digraph 

<img src = "images/dirgraph_sedgewick.png" width = "25%"/>

And here is how graphviz renders it. Which do you like better? 

<img src = "digraph_example.dot.png" width = "50%"/>

In [ ]:
order, prev = bfs(dg,'7')

## Example DiGraph

In [ ]:
dg = build_digraph('dg2.txt', sep=',')
gv = g2gviz(dg,'levitin4_2_9')
gv.render(filename='levitin4_2_9.dot')


<img src = "levitin4_2_9.dot.png" width = "25%"/>

## Reconstructing Paths from the `prev` dictionary

In [ ]:
def path(prev: Dict[T,T], x:T, y:T):
    s = []
    curr = y
    while curr != x:
        s.append(curr)
        curr = prev[curr]
    s.append(x)
    return s[::-1]


In [ ]:
print(path(prev, '7', '12'))

In [ ]:
prev

## Exercise - BFS
Do a BFS if the following graph starting at Boston and showing the state of the Queue at each step.

Also show the `prev` dictionary. Assume each city is a layover on a train ride. What path minimizes the number of layovers from Boston to Miami. Use abbreviations for city names like Bos, Det, LA

![USA](images/USA-unweighted.png)

In [ ]:
order,prev = bfs(g,'Boston')

In [ ]:
prev

## Shortest Path in an unweighted graph

In [ ]:
from collections import deque

def shortest_paths(g: Graph, v:T) -> Tuple[Dict[T,T], Dict[T,T]]:
    dist = dict()   # distance dictionary 
    q = deque()     # queue
    q.appendleft(v) # initialize queue
    prev = dict()   # previous dictionary
    dist[v] = 0     

    while len(q) > 0:
        print(q)
        curr = q.pop()        # removes right item in queue
        for e in g.get_edges(curr):
            if e not in dist:
                q.appendleft(e)
                dist[e] = dist[curr] + 1
                prev[e] = curr
    return dist, prev

In [ ]:
dist,prev = shortest_paths(g, 'Boston')

# Weighted Graphs

<img src="images/USA-weighted.png" />

The shortest path from Boston to Miami is not the path that visits the fewest vertices

```
Boston -> Detroit -> Washington -> Miami
```

but 

```
Boston -> New York -> Philadelphia -> Washington -> Miami
```

## Representing Weighted Graphs

Are they directed or undirected?

In [10]:
class WeightedGraph(Graph):
    def __init__(self):
        super().__init__()
        
    def add_edge(self, u: T, v: T, weight: float):
        if v not in self._vertices:
            self._vertices[v] = []
        if u not in self._vertices:
            self._vertices[u] = []
            
        self._vertices[v].append((u, weight))
        self._vertices[u].append((v, weight))

# Dikstra Algorithm

In [12]:
from math import inf
from typing import Set, Dict, TypeVar
T = TypeVar('T')

# s is the starting vertex
def sp(g: WeightedGraph, s: T):
    # dictionary comprehension
    dist = {v: inf for v in g.get_vertices()}
    dist[s] =  0
    seen = set() # region R from the textbook Dikstra explanation
    prev = dict()
    
    def smallest(dist: Dict[T, float], seen: Set[T]):
        for k in dist: # for each key in the dictionary  O(|V|)
            if k in seen:   # O(c)
                continue
                
            if dist[k] < smallest_dist:         # O(c)
                smallest_dist = dist[k]
                smallest_v = k
                
        return smallest_v
    # O(|V|)[O(|V|) + O(|E|)] = O(|V| ** 2 + )
    while len(seen) < len(g.get_vertices()):   # O(|V|)
        curr = smallest(dist, seen)
        seen.add(curr)
        for (e, w) in g.get_edges(curr):       # O(|E|)
            if dist[curr] + w < dist[e]:
                dist[e] = dist[curr] + w
                prev[e] = curr
                
    return dist, prev 